In [37]:
from modules import IO
import numpy as np
data_folder = './data/'
tmdb_genres_list = IO(data_folder + 'tmdb_genres_list.pkl').read_pickle()
movies = IO(data_folder + 'tmdb_processed.pkl').read_pickle()

In [38]:
movies = movies[130973:]

In [39]:
titles = np.array([m['title'] for m in movies])
plots = np.array([m['overview'] for m in movies])
genres = np.array([m['genre_ids'] for m in movies])

In [40]:
len(titles)

30000

In [41]:
k = len(tmdb_genres_list)
k

19

# Word2vec

In [42]:
from gensim import models
from gensim.models import Word2Vec

In [43]:
model2 = models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [44]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

In [45]:
movie_mean_wordvec=np.zeros((len(movies),300))
movie_mean_wordvec.shape

(30000, 300)

In [46]:
rows_to_delete=[]
for i in range(len(movies)):
    mov=movies[i]
    overview=mov['overview']
    tokens = tokenizer.tokenize(overview)
    stopped_tokens = [k for k in tokens if not k in en_stop]
    count_in_vocab=0
    s=0
    if len(stopped_tokens)==0:
        rows_to_delete.append(i)

    else:
        for tok in stopped_tokens:
            if tok.lower() in model2.vocab:
                count_in_vocab+=1
                s+=model2[tok.lower()]
        if count_in_vocab!=0:
            movie_mean_wordvec[i]=s/float(count_in_vocab)
        else:
            rows_to_delete.append(i)

In [47]:
mask2=[]
for row in range(len(movie_mean_wordvec)):
    if row in rows_to_delete:
        mask2.append(False)
    else:
        mask2.append(True)

In [48]:
X=movie_mean_wordvec[mask2]

In [49]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb=MultiLabelBinarizer()
Y=mlb.fit_transform(genres)

In [50]:
np.random.seed(10)
mask_text=np.random.rand(len(X))<0.8
X_train=X[mask_text]
Y_train=Y[mask_text]
X_test=X[~mask_text]
Y_test=Y[~mask_text]

In [8]:
#dimsize = 100

In [37]:
#X = np.array([m['overview'].split(" ") for m in movies])

In [47]:
# train model
#model = Word2Vec(X, iter=10, min_count=10, size=dimsize, workers=4)

In [33]:
#model.train(X, total_examples=len(X), epochs=10)

In [62]:
# def avg_word_vectors(wordlist,size):
#     """
#     returns a vector of zero for reviews containing words where none of them
#     met the min_count or were not seen in the training set
    
#     Otherwise return an average of the embeddings vectors
    
#     """
    
#     sumvec=np.zeros(shape=(1,size))
#     wordcnt=0
    
#     for w in wordlist:
#         if w in model:
#             sumvec += model.wv[w]
#             wordcnt +=1
    
#     if wordcnt ==0:
#         return sumvec
    
#     else:
#         return sumvec / wordcnt

In [34]:
# X_train = X[train_idx]
# X_test = X[test_idx]

In [76]:
# X_train=np.concatenate([avg_word_vectors(w,dimsize) for w in X_train_raw])
# X_test=np.concatenate([avg_word_vectors(w,dimsize) for w in X_test_raw])

/Users/gwungwun/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  


# Naive Bayes

In [28]:
def precision_recall(gt,preds):
    TP=0
    FP=0
    FN=0
    for i in range(len(gt)):
        if gt[i] ==1 and preds[i] == 1:
            TP+=1
        elif gt[i] ==0 and preds[i] == 1:
            FN+=1
        elif gt[i] ==1 and preds[i] == 0:
            FP+=1
    if TP+FP==0:
        precision=0
    else:
        precision=TP/float(TP+FP)
    if TP+FN==0:
        recall=0
    else:
        recall=TP/float(TP+FN)
    if TP + FP + FN == 0:
        f1 = 0
    else:
        f1 = 2 * TP / float(2 * TP + FP + FN)
    return precision,recall,f1

In [16]:
def hamming_loss(gt, preds):
    err = 0
    for i in range(len(gt)):
        if gt[i] ==0 and preds[i] == 1 or gt[i] ==1 and preds[i] == 0:
            err +=1
    return err

In [51]:
def evaluate(model, LabelPowerset = False, tfidf=False):
    if tfidf:
        X_tr = X_train_tfidf
        X_te = X_test_tfidf
    else:
        X_tr = X_train
        X_te = X_test
    
    model.fit(X_tr, Y_train)
#     train_acc = model.score(X_tr,Y_train)
#     test_acc = model.score(X_te,Y_test)
    
    if LabelPowerset:
        train_pred = model.predict(X_tr).toarray()
        test_pred = model.predict(X_te).toarray()
    else:
        train_pred = model.predict(X_tr)
        test_pred = model.predict(X_te)

    f1_micro_train = f1_score(Y_train, train_pred, average='micro') 
    f1_micro_test = f1_score(Y_test, test_pred, average='micro') 
    prec_micro_train = precision_score(Y_train, train_pred, average='micro') 
    prec_micro_test = precision_score(Y_test, test_pred, average='micro') 
    recs_micro_train = recall_score(Y_train, train_pred, average='micro') 
    recs_micro_test = recall_score(Y_test, test_pred, average='micro') 
        
#     train_precs=[]
#     train_recs=[]
#     test_precs=[]
#     test_recs=[]
#     train_f1=[]
#     test_f1=[]
# #     train_h_loss = []
# #     test_h_loss = []
    
#     for i in range(len(Y_test)):
#         a,b,c=precision_recall(Y_train[i],train_pred[i])
#         train_precs.append(a)
#         train_recs.append(b)
#         train_f1.append(c)
#         a,b,c=precision_recall(Y_test[i],test_pred[i])
#         test_precs.append(a)
#         test_recs.append(b)
#         test_f1.append(c)
#         train_h_loss.append(hamming_loss(Y_train[i],train_pred[i]))
#         test_h_loss.append(hamming_loss(Y_test[i],test_pred[i]))
    
    #print("Acc: {} (train)   {} (test)".format(train_acc, test_acc))
    print(" Training Precision:{} Recall :{} f1 micro score:{}" .format(prec_micro_train, recs_micro_train, f1_micro_train))
    print(" Test Precision:{} Recall :{} f1 micro score:{}" .format(prec_micro_test, recs_micro_test, f1_micro_test))
    #print(" Training Hamming Loss:{} Test Hamming Loss :{}" .format(np.mean(np.asarray(train_h_loss)/k), np.mean(np.asarray(test_h_loss)/k)))
    
    
    #print("F1: {} (train)   {} (test)".format(train_f1, test_f1))

In [52]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score, precision_score, recall_score
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.multiclass import OneVsRestClassifier
from skmultilearn.problem_transform import LabelPowerset
from sklearn.ensemble import RandomForestClassifier

In [53]:
from sklearn.svm import LinearSVC

In [54]:
svm = OneVsRestClassifier(LinearSVC())

In [55]:
evaluate(svm)

 Training Precision:0.6575953797689884 Recall :0.2253913982364585 f1 micro score:0.3357158811704266
 Test Precision:0.6193259972489684 Recall :0.21374317588416805 f1 micro score:0.3178048350097053


In [ ]:
1